In [ ]:
import cv2
import numpy as np
import pandas as pd

In [ ]:
valid_whole_image_dir = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"
valid_test_img = "Z107_Img11169.jpg"

In [ ]:
# read in list of truths
truth_file = "/data/old_home_dir/ChrissyF/GFRC/yolo_valid_GFRC_bboxes.csv"
truths = pd.read_csv(truth_file)
truths['filename'] = [strin.replace('/', '_') for strin in truths.file_loc]
truths

In [ ]:
truths_im = truths[truths.filename == valid_test_img]
truths_im

In [ ]:
# read in image
whole_im = cv2.imread(valid_whole_image_dir + valid_test_img)
whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)
whole_im.shape

In [ ]:
# split into tiles for yolo and keep list of split locations

def split_locations_array():
    # create list of tile splits
    gfrccolst = [0, 1856, 3712, 5504]
    gfrccolst = np.reshape(np.tile(gfrccolst, 4), (np.square(4), 1))
    gfrccoled = [1856, 3712, 5568, 7360]
    gfrccoled = np.reshape(np.tile(gfrccoled, 4), (np.square(4), 1))

    gfrcrowst = [0, 1248, 2496, 3664]
    gfrcrowst = np.reshape(np.repeat(gfrcrowst, 4), (np.square(4), 1))
    gfrcrowed = [1248, 2496, 3744, 4912]
    gfrcrowed = np.reshape(np.repeat(gfrcrowed, 4), (np.square(4), 1))

    gfrcwindz = np.hstack((gfrcrowst, gfrccolst, gfrcrowed, gfrccoled))
    gfrcwindz = np.array(gfrcwindz, dtype=np.int)
    
    return gfrcwindz

def create_tile_list(whole_im):
    gfrcwindz = split_locations_array()
    
    # split image into tiles
    im_tiles = []
    for tl in gfrcwindz:
        tile = whole_im[tl[0]:tl[2], tl[1]:tl[3]]
        im_tiles.append(tile)

    return im_tiles

In [ ]:
tilez = create_tile_list(whole_im)
    
from PIL import Image
Image.fromarray(tilez[0])

In [ ]:
# predict on each tile using yolo

In [ ]:
from scipy.special import expit
import torch
from torchvision import transforms, utils

from cnn_experiments.yolo.trch_weights import get_weights
from cnn_experiments.yolo.trch_yolonet import YoloNetOrig

In [ ]:
def yolo_output_to_box_test(y_pred, conf_threshold):    
    n_bat = y_pred.shape[0]
    # n_bat = int(dict_in['batch_size'])
    boxsx = y_pred.shape[2]
    # boxsx = int(dict_in['boxs_x'])
    boxsy = y_pred.shape[1]
    # boxsy = int(dict_in['boxs_y'])
    # anchors = dict_in['anchors']
    nanchors = anchors.shape[0]
    num_out = int(y_pred.shape[3] / nanchors)
    n_classes = num_out - 5
    # n_classes = int(dict_in['n_classes'])
    # num_out = 5 + n_classes
    # thresh = dict_in['threshold']
    # size of all boxes anchors and data
    size1 = [n_bat, boxsy, boxsx, nanchors, num_out]
    # size of all boxes and anchors
    size2 = [n_bat, boxsy, boxsx, nanchors]
    # number of boxes in each direction used for calculations rather than sizing so x first
    size3 = [boxsx, boxsy]
    # get top left position of cells
    rowz = np.arange(boxsy)
    colz = np.arange(boxsx)
    # rowno = np.reshape(np.repeat(np.repeat(rowz, boxsx * nanchors), n_bat), (n_bat, boxsy, boxsx, nanchors))
    rowno = np.expand_dims(np.expand_dims(np.reshape(np.repeat(rowz, boxsx), (boxsy, boxsx)), axis=0), axis=3)
    # colno = np.reshape(np.repeat(np.tile(np.repeat(colz, nanchors), boxsy), n_bat), (n_bat, boxsy, boxsx, nanchors))
    colno = np.expand_dims(np.expand_dims(np.reshape(np.tile(colz, boxsy), (boxsy, boxsx)), axis=0), axis=3)
    tl_cell = np.stack((colno, rowno), axis=4)
    # restructure net_output
    y_pred = np.reshape(y_pred, size1)

    # get confidences centres sizes and class predictions from from net_output
    confs_cnn = expit(np.reshape(y_pred[:, :, :, :, 4], size2))
    cent_cnn = expit(y_pred[:, :, :, :, 0:2])
    # cent_cnn_in = cent_cnn
    # add to cent_cnn so is position in whole image
    cent_cnn = np.add(cent_cnn, tl_cell)
    # divide so position is relative to whole image
    cent_cnn = np.divide(cent_cnn, size3)

    size_cnn = y_pred[:, :, :, :, 2:4]
    # size is to power of prediction
    size_cnn = np.exp(size_cnn)
    # keep for loss
    # size_cnn_in = size_cnn
    # adjust so size is relative to anchors
    size_cnn = np.multiply(size_cnn, anchors)
    # adjust so size is relative to whole image
    size_cnn = np.divide(size_cnn, size3)
    class_cnn = expit(y_pred[:, :, :, :, 5:])
    
    for img in range(n_bat):
        box_img = pd.DataFrame(columns=['xc', 'yc', 'wid', 'hei', 'conf', 'class'])
        for yc in range(boxsy):
            for xc in range(boxsx):
                for ab in range(nanchors):
                    if confs_cnn[img, yc, xc, ab] > conf_threshold:
                        # scores_out.append(confs_cnn[img, yc, xc, ab])
                        class_out = np.argmax(class_cnn[img, yc, xc, ab, :])
                        # classes_out.append(class_out)
                        detect_deets = pd.DataFrame(
                            [[
                                cent_cnn[img, yc, xc, ab, 0],
                                cent_cnn[img, yc, xc, ab, 1],
                                size_cnn[img, yc, xc, ab, 0],
                                size_cnn[img, yc, xc, ab, 1],
                                confs_cnn[img, yc, xc, ab],
                                class_out
                            ]],
                            columns=['xc', 'yc', 'wid', 'hei', 'conf', 'class']
                        )
                        box_img = box_img.append(detect_deets)
    
    return box_img


In [ ]:
img_w = 1856
img_h = 1248
max_annotations = 14
anchors = [[2.387088, 2.985595], [1.540179, 1.654902], [3.961755, 3.936809], [2.681468, 1.803889], [5.319540, 6.116692]]
nclazz = 1
box_size = [32, 32]
weightspath = "/data/old_home_dir/ChrissyF/GFRC/yolov2.weights"
saveweightspath = "/home/cmf21/pytorch_save/GFRC/Bin/rgb_baseline2/testing_save_163.pt"
lambda_c = 5.0
lambda_no = 0.5
lambda_cl = 1
lambda_cf = 1
n_box = 5
out_len = 5 + nclazz
fin_size = n_box * out_len
grid_w = int(img_w / box_size[1])
grid_h = int(img_h / box_size[0])
input_vec = [grid_w, grid_h, n_box, out_len]
anchors = np.array(anchors)
channels_in = 3
conf_threshold = 0.3

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
layerlist = get_weights(weightspath)
net = YoloNetOrig(layerlist, fin_size, channels_in)
net = net.to(device)
net.load_state_dict(torch.load(saveweightspath))
net.eval()

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms

class TileImageTestDataset(Dataset):

    def __init__(self, tiles_list):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.landmarks_frame = pd.read_csv(csv_file)
        self.tiles_list = tiles_list
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.tiles_list)

    def __getitem__(self, idx):
        image = self.tiles_list[idx]
        image = self.transform(image)

        return image

In [ ]:
from torch.utils.data import DataLoader

tile_dataset = TileImageTestDataset(tilez)
tileloader = DataLoader(tile_dataset, batch_size=1, shuffle=False)
boxes_whole_im = pd.DataFrame(columns=['xc', 'yc', 'wid', 'hei', 'conf', 'class', 'tile'])
for idx, tile in enumerate(tileloader):
    tile = tile.to(device)
    y_pred = net(tile)
    y_pred_np = y_pred.data.cpu().numpy()
    boxes_tile = yolo_output_to_box_test(y_pred_np, conf_threshold)
    boxes_tile['tile'] = idx
    boxes_whole_im = pd.concat((boxes_whole_im, boxes_tile), axis=0)

In [ ]:
def process_annotation_df_negative(detections_fil, img_size):
    pix_width, pix_height = img_size
    detections_fil['wid_pixels'] = np.array(np.multiply(detections_fil['wid'], pix_width), dtype=np.int)
    detections_fil['hei_pixels'] = np.array(np.multiply(detections_fil['hei'], pix_height), dtype=np.int)
    detections_fil['square_size'] = np.maximum(detections_fil['wid_pixels'], detections_fil['hei_pixels'])
    detections_fil['xc_pix'] = np.array(np.multiply(detections_fil['xc'], pix_width), dtype=np.int)
    detections_fil['yc_pix'] = np.array(np.multiply(detections_fil['yc'], pix_height), dtype=np.int)
    detections_fil['xmin'] = np.array(np.minimum(np.maximum(np.subtract(detections_fil['xc_pix'], np.divide(detections_fil['square_size'], 2)), 0), np.subtract(pix_width, detections_fil['square_size'])), dtype=np.int)
    detections_fil['ymin'] = np.array(np.minimum(np.maximum(np.subtract(detections_fil['yc_pix'], np.divide(detections_fil['square_size'], 2)), 0), np.subtract(pix_height, detections_fil['square_size'])), dtype=np.int)
    detections_fil['xmax'] = np.add(detections_fil['xmin'], detections_fil['square_size'])
    detections_fil['ymax'] = np.add(detections_fil['ymin'], detections_fil['square_size'])
    detections_fil = detections_fil.reset_index()
    return detections_fil

def create_windows_from_yolo(windows_in, tile_list):
    windowz_out = []
    for index, row in windows_in.iterrows():
        tile_for_row = tile_list[row.tile]
        row_array = tile_for_row[row.ymin:row.ymax, row.xmin:row.xmax]
        row_pil = Image.fromarray(row_array)
        windowz_out.append(row_pil)
    return windowz_out


In [ ]:
# convert output positions to windows

windows_whole_im = process_annotation_df_negative(boxes_whole_im, [img_w, img_h])

windows_list = create_windows_from_yolo(windows_whole_im, tilez)

windows_list[0]

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import loggers as pl_loggers
from torchvision.models import inception_v3

# predict on windows
class BinaryWindowClassifier(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()
        self.model = inception_v3(num_classes=2)

    def training_step(self, batch, batch_idx):
        x, y = batch
        output, aux1 = self.model(x)
        pred = torch.log_softmax(output, dim=1)

        criterion = nn.CrossEntropyLoss()
        loss1 = criterion(output, y)
        loss2 = criterion(aux1, y)
        ### TODO:  check how losses are weighted
        loss = loss1 + 0.4 * loss2
        self.log("train_loss", loss)
        
        correct=pred.argmax(dim=1).eq(y).sum().item()
        total=len(y)   
        accu = correct / total
        self.log("train_accuracy", accu)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        output = self.model(x)
        pred = torch.log_softmax(output, dim=1)

        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, y)
        self.log("val_loss", loss)
        
        correct=pred.argmax(dim=1).eq(y).sum().item()
        total=len(y)   
        accu = correct / total
        self.log("val_accuracy", accu)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters())
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5),
            'interval': 'step' 
        }
        return [optimizer], [scheduler]
    
    def forward(self, x):
        return self.model(x)
        
    
cp_path = "/home/cmf21/pytorch_save/GFRC/Bin/rgb_baseline2_post/patch_model/checkpoint.ckpt.ckpt"
classifier = BinaryWindowClassifier.load_from_checkpoint(checkpoint_path=cp_path)
classifier.eval()
classifier.to(device)

In [ ]:
class WindowTestDataset(Dataset):

    def __init__(self, windows_list):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.landmarks_frame = pd.read_csv(csv_file)
        self.windows_list = windows_list
        self.transform = transforms.Compose([Resize((299, 299)),
                                             ToTensor(),
                                             Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    def __len__(self):
        return len(self.windows_list)

    def __getitem__(self, idx):
        image = self.windows_list[idx]
        image = self.transform(image)

        return image

In [ ]:
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

window_dataset = WindowTestDataset(windows_list)
windowloader = DataLoader(window_dataset, batch_size=len(window_dataset), shuffle=False)

len(windowloader)

In [ ]:
for batch in windowloader:
    batch = batch.to(device)
    output = classifier(batch)
    sm = torch.nn.Softmax(1)
    output_sm = sm(output)
    pred_prob = output_sm.cpu().detach().numpy()

pred_prob

In [ ]:
# save list of positive windows
preds_df = pd.DataFrame(pred_prob, columns=['animal', 'not_animal'])
windows_with_preds = pd.concat((windows_whole_im, preds_df), axis=1)
windows_filter_out = windows_with_preds[windows_with_preds.animal > 0.5]
windows_filter_out

In [ ]:
def windows_truth(df_in, img_sz):
    df_in['xc_pix_rect'] = np.multiply(df_in['xc'], img_sz[1])
    df_in['yc_pix_rect'] = np.multiply(df_in['yc'], img_sz[0])
    df_in['wid_rect'] = np.multiply(df_in['wid'], img_sz[1])
    df_in['hei_rect'] = np.multiply(df_in['height'], img_sz[0])
    df_in['xmn'] = np.array(np.subtract(df_in['xc_pix_rect'], np.divide(df_in['wid_rect'], 2)), dtype=np.int)
    df_in['xmx'] = np.array(np.add(df_in['xc_pix_rect'], np.divide(df_in['wid_rect'], 2)), dtype=np.int)
    df_in['ymn'] = np.array(np.subtract(df_in['yc_pix_rect'], np.divide(df_in['hei_rect'], 2)), dtype=np.int)
    df_in['ymx'] = np.array(np.add(df_in['yc_pix_rect'], np.divide(df_in['hei_rect'], 2)), dtype=np.int)
    return df_in

def windows_out(df_in, tile_w, tile_h, img_w, img_h):
    tile_vals = split_locations_array()
    xmin = np.subtract(df_in['xc'], np.divide(df_in['wid'], 2))
    xmax = np.add(df_in['xc'], np.divide(df_in['wid'], 2))
    ymin = np.subtract(df_in['yc'], np.divide(df_in['hei'], 2))
    ymax = np.add(df_in['yc'], np.divide(df_in['hei'], 2))
    xmin = np.multiply(xmin, tile_w)
    xmax = np.multiply(xmax, tile_w)
    ymin = np.multiply(ymin, tile_h)
    ymax = np.multiply(ymax, tile_h)
    tileord = df_in.tile.tolist()
    tilerowst = tile_vals[tileord, 0]
    tilecolst = tile_vals[tileord, 1]
    xmin = np.add(xmin, tilecolst)
    xmax = np.add(xmax, tilecolst)
    ymin = np.add(ymin, tilerowst)
    ymax = np.add(ymax, tilerowst)
    xmin = np.maximum(xmin, 0)
    xmax = np.minimum(xmax, img_w)
    ymin = np.maximum(ymin, 0)
    ymax = np.minimum(ymax, img_h)
    df_in['xmn'] = np.array(xmin, dtype=np.int)
    df_in['xmx'] = np.array(xmax, dtype=np.int)
    df_in['ymn'] = np.array(ymin, dtype=np.int)
    df_in['ymx'] = np.array(ymax, dtype=np.int)
    return df_in


In [ ]:
# compare positive windows to trut

whole_im_size = [4912, 7360]

truths_pixels = windows_truth(truths, whole_im_size)

In [ ]:
print([img_h, img_w])
windows_out_pixels = windows_out(windows_filter_out, img_w, img_h, whole_im_size[1], whole_im_size[0])
windows_out_pixels[['xmn', 'xmx', 'ymn', 'ymx']]

In [ ]:
# filter truths
truths_per_im = truths[truths.filename == valid_test_img]
truths_per_im.shape

In [ ]:
columns_out = ['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal', 'confmat', 'tru_box']
results_out = pd.DataFrame(columns=columns_out)

if truths_per_im.shape[0] == 0:
    results_per_im = windows_out_pixels[['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal']]
    results_per_im['confmat'] = 'FP'
    results_per_im['tru_box'] = ''
    results_out = pd.concat((results_out, results_per_im), axis=0)
    
#results_out = results_out.reset_index()
results_out

In [ ]:
def intersection_over_union(box1, box2):
    # determine the (x, y)-coordinates of the intersection rectangle
    xa = max(box1.xmn, box2.xmn)
    xb = min(box1.xmx, box2.xmx)
    ya = max(box1.ymn, box2.ymn)
    yb = min(box1.ymx, box2.ymx)
    # compute the area of intersection rectangle
    inter_area = max(0, xb - xa + 1) * max(0, yb - ya + 1)
    # compute the area of both the prediction and ground-truth
    box1_area = (box1.xmx - box1.xmn + 1) * (box1.ymx - box1.ymn + 1)
    box2_area = (box2.xmx - box2.xmn + 1) * (box2.ymx - box2.ymn + 1)
    # compute the intersection over union 
    iou = inter_area / float(box1_area + box2_area - inter_area)
    # return the intersection over union value
    return iou

def nms_for_fp(boxes_in, thresh):
    
    xmins = boxes_in.xmn
    xmaxs = boxes_in.xmx
    ymins = boxes_in.ymn
    ymaxs = boxes_in.ymx
    confs = boxes_in.conf
    anims = boxes_in.animal
    notas = boxes_in.not_animal

    boxes_ot = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal'])

    xmins = np.array(xmins.tolist())
    xmaxs = np.array(xmaxs.tolist())
    ymins = np.array(ymins.tolist())
    ymaxs = np.array(ymaxs.tolist())
    confs = np.array(confs.tolist())
    anims = np.array(anims.tolist())
    notas = np.array(notas.tolist())

    while len(xmins) > 0:

        xmn = xmins[0]
        xmns = np.array(xmins[1:])
        xmx = xmaxs[0]
        xmxs = np.array(xmaxs[1:])
        ymn = ymins[0]
        ymns = np.array(ymins[1:])
        ymx = ymaxs[0]
        ymxs = np.array(ymaxs[1:])
        cnf = confs[0]
        cnfs = np.array(confs[1:])
        ani = anims[0]
        anis = np.array(anims[1:])
        ntz = notas[0]
        nots = np.array(notas[1:])

        ol_wid = np.minimum(xmx, xmxs) - np.maximum(xmn, xmns)
        ol_hei = np.minimum(ymx, ymxs) - np.maximum(ymn, ymns)

        ol_x = np.maximum(0, ol_wid)
        ol_y = np.maximum(0, ol_hei)

        distx = np.subtract(xmxs, xmns)
        disty = np.subtract(ymxs, ymns)
        bxx = xmx - xmn
        bxy = ymx - ymn

        ol_area = np.multiply(ol_x, ol_y)
        bx_area = bxx * bxy
        bxs_area = np.multiply(distx, disty)

        ious = np.divide(ol_area, np.subtract(np.add(bxs_area, bx_area), ol_area))
        mask_bxs = np.greater(ious, thresh)

        if np.sum(mask_bxs) > 0:
            box_ot = pd.DataFrame(index=[1], columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal'])
            xmns = xmns[mask_bxs]
            xmxs = xmxs[mask_bxs]
            ymns = ymns[mask_bxs]
            ymxs = ymxs[mask_bxs]
            cnfs = cnfs[mask_bxs]
            anis = anis[mask_bxs]
            nots = nots[mask_bxs]

            box_ot.xmn.iloc[0] = np.min(xmns)
            box_ot.ymn.iloc[0] = np.min(ymns)
            box_ot.xmx.iloc[0] = np.max(xmxs)
            box_ot.ymx.iloc[0] = np.max(ymxs)
            box_ot.conf.iloc[0] = np.mean(cnfs)
            box_ot.animal.iloc[0] = np.mean(anis)
            box_ot.not_animal.iloc[0] = np.mean(nots)

            mask_out = np.repeat(False, len(xmins))
            mask_out[0] = True
            mask_out[1:] = mask_bxs
            mask_out = np.logical_not(mask_out)

            xmins = xmins[mask_out]
            xmaxs = xmaxs[mask_out]
            ymins = ymins[mask_out]
            ymaxs = ymaxs[mask_out]
            confs = confs[mask_out]
            anims = anims[mask_out]
            notas = notas[mask_out]
        else:
            box_ot = pd.DataFrame(index=[1], columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal'])
            box_ot.xmn.iloc[0] = xmn
            box_ot.ymn.iloc[0] = ymn
            box_ot.xmx.iloc[0] = xmx
            box_ot.ymx.iloc[0] = ymx
            box_ot.conf.iloc[0] = cnf
            box_ot.animal.iloc[0] = ani
            box_ot.not_animal.iloc[0] = ntz

            mask_out = np.repeat(False, len(xmins))
            mask_out[0] = True
            mask_out = np.logical_not(mask_out)
            
            xmins = xmins[mask_out]
            xmaxs = xmaxs[mask_out]
            ymins = ymins[mask_out]
            ymaxs = ymaxs[mask_out]
            confs = confs[mask_out]
            anims = anims[mask_out]
            notas = notas[mask_out]
        boxes_ot = pd.concat((boxes_ot, box_ot), axis=0)
        
    boxes_ot['confmat'] = 'FP'
    boxes_ot['tru_box'] = ''

    return boxes_ot
 

iou_threshold = 0.15

def match_truths(windows_out_pixels, truths_per_im):

    results_out = pd.DataFrame(columns = ['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal'])

    if truths_per_im.shape[0] > 0:
        results_per_im = windows_out_pixels[['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'animal', 'not_animal']]
        truz = [True] * truths_per_im.shape[0]
        rez = [True] * results_per_im.shape[0]
        matchz = np.array([True] * results_per_im.shape[0])
        for idx, tru in truths_per_im.iterrows():
            iouz = []
            for res_idx, result in results_per_im.iterrows():
                iou = intersection_over_union(tru, result)
                iouz.append(iou)
            iou_ind = np.argmax(iouz)
            if iouz[iou_ind] > iou_threshold:
                if rez[iou_ind]: 
                    best_iou_res = results_per_im.iloc[iou_ind:(iou_ind+1), :]
                    best_iou_res['confmat'] = 'TP'
                    true_box = f'xmin: {tru.xmn}, xmax:{tru.xmx}, ymin: {tru.ymn}, ymax: {tru.ymx}'
                    best_iou_res['tru_box'] = true_box
                    results_out = pd.concat((results_out, best_iou_res), axis=0, ignore_index=True)
                    truz[idx] = False
                    rez[iou_ind] = False
            # matchz removes any matches that overlap but are not the most overlapping
            match_mask = np.array(iouz) > iou_threshold
            matchz[match_mask] = False

        results_per_im = results_per_im[matchz]
        results_per_im['confmat'] = 'FP'
        results_per_im['tru_box'] = ''
        results_per_im = nms_for_fp(results_per_im, 0.5)
        results_out = pd.concat((results_out, results_per_im), axis=0, ignore_index=True)
        false_negatives = truths_per_im[['xmn', 'xmx', 'ymn', 'ymx']]
        false_negatives = false_negatives[truz]
        false_negatives['conf'] = 0
        false_negatives['animal'] = 0
        false_negatives['not_animal'] = 0
        false_negatives['confmat'] = 'FN'
        false_negatives['tru_box'] = ''
        results_out = pd.concat((results_out, false_negatives), axis=0, ignore_index=True)

    return results_out   

results_out = match_truths(windows_out_pixels, truths_per_im)

results_out

In [ ]:
# write on image
import copy

whole_im_out = copy.deepcopy(whole_im)

# draw TP
tp_results = results_out[results_out.confmat == 'TP']
for idx, row in tp_results.iterrows():
    cv2.rectangle(whole_im_out,(row.xmn,row.ymn),(row.xmx,row.ymx),(0,255,0),5)
    
# draw FP
fp_results = results_out[results_out.confmat == 'FP']
for idx, row in fp_results.iterrows():
    cv2.rectangle(whole_im_out,(row.xmn,row.ymn),(row.xmx,row.ymx),(255,255,0),5)
    
# draw FN
fn_results = results_out[results_out.confmat == 'FN']
for idx, row in fn_results.iterrows():
    cv2.rectangle(whole_im_out,(row.xmn,row.ymn),(row.xmx,row.ymx),(255,0,0),5)   
    
    
Image.fromarray(whole_im_out)



In [ ]:
windows_yolo_pixels = windows_out(windows_whole_im, img_w, img_h, whole_im_size[1], whole_im_size[0])
windows_yolo_pixels['animal'] = 0
windows_yolo_pixels['not_animal'] = 0
yolo_out_wind = match_truths(windows_yolo_pixels, truths_per_im)

yolo_out_wind

In [ ]:
windows_with_preds

In [ ]:
yolo_im_out = copy.deepcopy(whole_im)

# draw TP
tp_results = yolo_out_wind[yolo_out_wind.confmat == 'TP']
for idx, row in tp_results.iterrows():
    cv2.rectangle(yolo_im_out,(row.xmn,row.ymn),(row.xmx,row.ymx),(0,255,0),5)
    
# draw FP
fp_results = yolo_out_wind[yolo_out_wind.confmat == 'FP']
for idx, row in fp_results.iterrows():
    cv2.rectangle(yolo_im_out,(row.xmn,row.ymn),(row.xmx,row.ymx),(255,255,0),5)
    
# draw FN
fn_results = yolo_out_wind[yolo_out_wind.confmat == 'FN']
for idx, row in fn_results.iterrows():
    cv2.rectangle(yolo_im_out,(row.xmn,row.ymn),(row.xmx,row.ymx),(255,0,0),5)   
    
    
Image.fromarray(yolo_im_out)